# AIS Drilldown

This notebook is work in progress that declares a Panel dashboard to visualize AIS data. All available AIS data is visualized with datashader and a `DatetimeInput` panel widget allows display of vessel locations at a chosen time. Tapping on a vessel shows more information in a drilldown table.

Not yet optimized and filtering vessels can take a few moments.

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import param
from colorcet import fire
import holoviews as hv
from holoviews.operation.datashader import rasterize
hv.extension('bokeh')

In [ ]:
mapping = {
    0:'Not available',
    20:'Wing in ground (WIG), all ships of this type',
    21:'Wing in ground (WIG), Hazardous category A',
    22:'Wing in ground (WIG), Hazardous category B',
    23:'Wing in ground (WIG), Hazardous category C',
    24:'Wing in ground (WIG), Hazardous category D',
    25:'Wing in ground (WIG), Reserved for future use',
    26:'Wing in ground (WIG), Reserved for future use',
    27:'Wing in ground (WIG), Reserved for future use',
    28:'Wing in ground (WIG), Reserved for future use',
    29:'Wing in ground (WIG), Reserved for future use',
    30:'Fishing',
    31:'Towing',
    32:'Towing: length exceeds 200m or breadth exceeds 25m',
    33:'Dredging or underwater ops',
    34:'Diving ops',
    35:'Military ops',
    36:'Sailing',
    37:'Pleasure Craft',
    38:'Reserved',
    39:'Reserved',
    40:'High speed craft (HSC), all ships of this type',
    41:'High speed craft (HSC), Hazardous category A',
    42:'High speed craft (HSC), Hazardous category B',
    43:'High speed craft (HSC), Hazardous category C',
    44:'High speed craft (HSC), Hazardous category D',
    45:'High speed craft (HSC), Reserved for future use',
    46:'High speed craft (HSC), Reserved for future use',
    47:'High speed craft (HSC), Reserved for future use',
    48:'High speed craft (HSC), Reserved for future use',
    49:'High speed craft (HSC), No additional information',
    50:'Pilot Vessel',
    51:'Search and Rescue vessel',
    52:'Tug',
    53:'Port Tender',
    54:'Anti-pollution equipment',
    55:'Law Enforcement',
    56:'Spare - Local Vessel',
    57:'Spare - Local Vessel',
    58:'Medical Transport',
    59:'Noncombatant ship according to RR Resolution No. 18',
    60:'Passenger, all ships of this type',
    61:'Passenger, Hazardous category A',
    62:'Passenger, Hazardous category B',
    63:'Passenger, Hazardous category C',
    64:'Passenger, Hazardous category D',
    65:'Passenger, Reserved for future use',
    66:'Passenger, Reserved for future use',
    67:'Passenger, Reserved for future use',
    68:'Passenger, Reserved for future use',
    69:'Passenger, No additional information',
    70:'Cargo, all ships of this type',
    71:'Cargo, Hazardous category A',
    72:'Cargo, Hazardous category B',
    73:'Cargo, Hazardous category C',
    74:'Cargo, Hazardous category D',
    75:'Cargo, Reserved for future use',
    76:'Cargo, Reserved for future use',
    77:'Cargo, Reserved for future use',
    78:'Cargo, Reserved for future use',
    79:'Cargo, No additional information',
    80:'Tanker, all ships of this type',
    81:'Tanker, Hazardous category A',
    82:'Tanker, Hazardous category B',
    83:'Tanker, Hazardous category C',
    84:'Tanker, Hazardous category D',
    85:'Tanker, Reserved for future use',
    86:'Tanker, Reserved for future use',
    87:'Tanker, Reserved for future use',
    88:'Tanker, Reserved for future use',
    89:'Tanker, No additional information',
    90:'Other Type, all ships of this type',
    91:'Other Type, Hazardous category A',
    92:'Other Type, Hazardous category B',
    93:'Other Type, Hazardous category C',
    94:'Other Type, Hazardous category D',
    95:'Other Type, Reserved for future use',
    96:'Other Type, Reserved for future use',
    97:'Other Type, Reserved for future use',
    98:'Other Type, Reserved for future use',
    99:'Other Type, no additional information'}

def vessel_type_from_int(val):
    if np.isnan(val):
        return 'Unknown vessel type'
    name = mapping.get(val, None)
    if name is not None:
        return name
    if val < 20:
        return 'Reserved for future use'
    else:
        return 'Unknown vessel type'

In [ ]:
zone1 = pd.read_csv('./data/AIS_2017_01_Zone01.csv', parse_dates=[1])
zone2 = pd.read_csv('./data/AIS_2017_01_Zone02.csv', parse_dates=[1])
zone3 = pd.read_csv('./data/AIS_2017_01_Zone03.csv', parse_dates=[1])
zones = pd.concat([zone1,zone2, zone3])

In [ ]:
zones.head() # Dataframe structure

In [ ]:
vessels = {name:df.drop_duplicates().sort_values(by='BaseDateTime').set_index('BaseDateTime') for name,df in zones.groupby('VesselName')}

In [ ]:
# NOTE! Some of the data is missing (presumably an unknown vessel)
# Consider substituting MMSI for VesselName if missing
zone1.iloc[8859]

In [ ]:
print("Expected Latitude range %.3f to %.3f"% (min(zones['LAT']), max(zones['LAT'])))
print("Expected Longitude range %.3f to %.3f " % (min(zones['LON']), max(zones['LON'])))

In [ ]:
columns = list([el for el in zones.columns if el!= 'BaseDateTime'])

In [ ]:
eastings, northings = zip(*[hv.util.transform.lon_lat_to_easting_northing(lon, lat) for lon, lat 
                          in zip(zones['LON'], zones['LAT'])])

In [ ]:
dt_input = pn.widgets.DatetimeInput(name='Datetime', value=pd.Timestamp('2017-01-18'))
dt_input

In [ ]:
table_cols = ['MMSI', 'VesselName', 'VesselType', 'Heading', 'CallSign', 'Length', 'Width', 'Cargo']
empty_df = pd.DataFrame({el:[] for el in table_cols})

class Drilldown(param.Parameterized):
    selection = param.DataFrame(empty_df)
    
    @param.depends('selection')
    def update_table(self, *args, **kwargs):
        return pn.widgets.DataFrame(self.selection, show_index=False)
        
    
drilldown = Drilldown()

In [ ]:
def vessel_at_time(vessel_name, time, vessels):
    df = vessels[vessel_name].drop_duplicates()
    if time < df.index[0]:
        return None # Query before first value
    if time > df.index[-1]:
        return None # Query after last value
    try:
        idx = df.index.get_loc(time, method='nearest')
        return df.iloc[idx]
    except:
        return None

marked_points = None # TODO: Declare a class and make this an attribute

def mark_vessels(value):
    global marked_points
    records = []
    empty = dict({'easting':0., 'northing':-7.081154551613623e-10}, **{col:'' for col in columns})
    for vessel in vessels.keys():
        match = vessel_at_time(vessel, value, vessels)
        if match is not None:
            easting, northing = hv.util.transform.lon_lat_to_easting_northing(match['LON'], match['LAT'])
            records.append(dict({'easting':easting, 'northing':northing}, **{col:match[col] for col in columns}))
    markers = pd.DataFrame(records if len(records) != 0 else [empty]) 
    alpha = 1 if len(records) else 0
    marked_points = hv.Points(markers, ['easting', 'northing'], columns).opts(color='white', size=4, 
                                                                                 marker='triangle', alpha=alpha)
    return marked_points

In [ ]:
def markerfn(index, table_cols=table_cols):
    if len(index) > 0:
        rows = [marked_points.data.iloc[ind] for ind in index]
        
        print(rows)
        df = pd.DataFrame(rows)[table_cols]
        print(df.columns)
        df.columns = table_cols
        df['VesselType'] = df['VesselType'].apply(vessel_type_from_int)
        drilldown.selection = df

    return hv.HLine(0).opts(visible=False)

In [ ]:
points = rasterize(hv.Points(pd.DataFrame({'northing':northings, 
                                           'easting':eastings}), ['easting', 'northing']))
tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')
dmap = hv.DynamicMap(mark_vessels, streams=[dt_input.param.value])
overlay = (tiles * points.opts(cmap=fire[180:], width=900, height=500, cnorm='eq_hist', alpha=0.5) * dmap)

marker = hv.DynamicMap(markerfn, streams=[hv.streams.Selection1D(source=dmap)]).opts(tools=['tap'])

message = ("Example times to compare: 2017-01-18 00:00:00 with 2017-01-17 00:00:00 (first sample) "
           "white triangles marking vessel locations. Unoptimized first cut: filtering updates can take a few moments.")
pn.Column('# AIS Data (Work in progress)', message, dt_input, overlay * marker, drilldown.update_table).servable()